# TRABALHO PRÁTICO 2 - GRUPO 14

## Problema 3 - KYBER
Este problema consistia em implementar o algoritmo KEM que seja IND-CPA seguro e num algoritmo PKE que seja IND-CCA seguro para a técnica pós-quântica baseada em reticulados, **KYBER**. Contudo, a implementação do KYBER apresentada no paper [KYBER](https://www.dropbox.com/sh/h1dsk0q09jrfz5u/AACGjdrn78b4kA0uuGaYF1IGa/KYBER-20201001/Supporting_Documentation?dl=0&preview=kyber.pdf&subfolder_nav_tracking=1) corresponde às etapas necessárias para implementar o PKE-IND-CPA e um KEM-IND-CCA. Para conseguir implementar um KEM-IND-CPA irá se transformar o PKE-IND-CPA num KEM tal como está apresentado na secção "Resolução do problema - KEM (IND-CPA)". Para se transformar um PKE-IND-CPA num PKE-IND-CCA aplica-se as transformações de Fujisaki-Okamoto tal como está nos [apontamentos](https://paper.dropbox.com/doc/Capitulo-2a-Esquemas-de-Cifra-Construcao-lpfWfmLxl3jNOqaAt4HgB)

#### **IMPORTS**

In [175]:
import random
import numpy as np
from cryptography.hazmat.primitives import hashes
from pickle import dumps, loads

#### **CLASSES AUXILIARES**

**NTT** Nesta classe fornecidas pelos docentes são implementados as operações designadas por *number-theoretic transform* (NTT) que permite realizar operações de multiplicação no anel Rq de forma eficiente. Por simplicidade estes esta classe é inicializada com os valores n=256 e q=7681. 

In [176]:
#Classe que implementa as multiplicações em R - number-theoretic transform (NTT) 
class NTT:

    def __init__(self, n=128, q=None):
        
        if not  n in [32,64,128,256,512,1024,2048]:
            raise ValueError("improper argument ",n)
        self.n = n  
        if not q:
            self.q = 1 + 2*n
            while True:
                if (self.q).is_prime():
                    break
                self.q += 2*n
        else:
            if q % (2*n) != 1:
                raise ValueError("Valor de 'q' não verifica a condição NTT")
            self.q = q
             
        self.F = GF(self.q) ;  self.R = PolynomialRing(self.F, name="w")
        w = (self.R).gen()
        
        g = (w^n + 1)
        xi = g.roots(multiplicities=False)[-1]
        self.xi = xi
        rs = [xi^(2*i+1)  for i in range(n)] 
        self.base = crt_basis([(w - r) for r in rs]) 
    
    def ntt(self,f):
        
        def _expand_(f): 
            u = f.list()
            return u + [0]*(self.n-len(u)) 
        
        def _ntt_(xi,N,f):
            if N==1:
                return f
            N_ = N/2 ; xi2 =  xi^2  
            f0 = [f[2*i]   for i in range(N_)] ; f1 = [f[2*i+1] for i in range(N_)] 
            ff0 = _ntt_(xi2,N_,f0) ; ff1 = _ntt_(xi2,N_,f1)  
    
            s  = xi ; ff = [self.F(0) for i in range(N)] 
            for i in range(N_):
                a = ff0[i] ; b = s*ff1[i]  
                ff[i] = a + b ; ff[i + N_] = a - b 
                s = s * xi2                     
            return ff
        
        return _ntt_(self.xi,self.n,_expand_(f))
        
    def invNtt(self,ff):                             
        return sum([ff[i]*self.base[i] for i in range(self.n)])

**MyMatrix** Nesta classe são implementados as operações sobre matrizes e vetores. É de notar que ao longo da explicação da implementação do KYBER todos os cálculos entre elementos de matrizes e vetores serão realizadas com recurso a esta classe. 

In [177]:
#Classe que implementa as operações sobre matrizes e vetores
class MyMatrix:
    
    #Soma entre matrizes
    def sumMatrix(self,e1,e2,n) :
    
        for i in range(len(e1)):
            e1[i] = self.sumValue(e1[i], e2[i],n)
            
        return e1
    
    #Subtração entre matrizes
    def subMatrix(self,e1,e2,n) :
    
        for i in range(len(e1)):
            e1[i] = self.subValue(e1[i], e2[i],n)
            
        return e1
    
    #Multiplicação entre matrizes
    def multMatrix(self,vec1, vec2, n):
        
        for i in range(len(vec1)):
            vec1[i] = self.multValue(vec1[i], vec2[i],n)
    
        tmp = [0] * n
        for i in range(len(vec1)):
            tmp = self.sumValue(tmp, vec1[i],n)
            
        return tmp
    
    #Multiplicação entre matriz e vector
    def multMatrixVector(self,M,v,k,n) :
        for i in range(len(M)):
            for j in range(len(M[i])):
                M[i][j] = self.multValue(M[i][j], v[j],n)
    
        tmp = [[0] * n] * k 
        for i in range(len(M)):
            for j in range(len(M[i])):
                tmp[i] = self.sumValue(tmp[i], M[i][j],n)
    
        return tmp
    
    #Soma elementos entre dois vetores
    def sumValue(self,ff1, ff2,n):
        
        res = []
    
        for i in range(n):
            res.append((ff1[i] + ff2[i]))

        return res
    
    #Multiplica elementos entre dois vetores
    def multValue(self,ff1, ff2,n):
        
        res = []
    
        for i in range(n):
            res.append((ff1[i] * ff2[i]))

        return res
    
    #Subtrai elementos entre dois vetores
    def subValue(self,ff1, ff2,n):
        
        res = []
    
        for i in range(n):
            res.append((ff1[i] - ff2[i]))

        return res

### Resolução do Problema

### Implementação PKE (IND-CPA)

Na classe abaixo é implementado o algoritmo PKE-IND-CPA, segundo a especificação que se encontra no paper, que irá cifrar uma informação aleatória gerada a partir do anel Rq dado por $F2[X]/(X^r + 1)$. Esta classe irá ser utilizada, posteriormente, para conseguir implementar um KEM-IND-CPA e um PKE-IND-CCA, tal como é pedido no enunciado. Assim sendo, será necessário implementar 3 funcionalidades principais:

**Geração do par de chaves:** Esta secção foi implementada segundo o algoritmo 4 apresentado no paper que vai gerar uma chave privada e outra pública para serem usadas no *encrypt* e *decrypt*, através da função `keyGen`. Este algoritmo começa por determinar a matriz $A ∈ Rq$ no dominio NTT e os vetores $e, s ∈ Rq$. Para isso utilizou-se as funções `Parse`, `CBD` que foram implementadas com recurso ao algoritmos 1 e 2, respetivamente. Para além desta funções também foram implementadas as funções de hash `XOF`, `G`, `PRF` que utilizam, respetivamente, Shake-128, Sha3-356 e Shake-256 tal como está previsto na documentação. As variáveis geradas são depois utilizadas para calcular a chave pública que será dada por $pk = (A * s + e, \rho)$, com  $\rho = G(d)[32:]$ e d = valor pseudo-aleatório. A chave privada será dada por $sk = s$.

**Cifra:** A função `encrypt` foi implementada com recurso ao algoritmo 5 do paper que permite gerar um ciphertext de uma mensagem *m* gerada a partir do anel *Rq*, utilizando para isso a chave pública e um valor *coins* gerado aleatóriamente. Esta função começa por gerar novamente a matriz $A^T ∈ Rq$ no dominio NTT e os vetores $e1, r ∈ Rq$ para além do vetor $e2 ∈ Rq$ que serão utilizadas para calcular as duas componetes do ciphertext, *c1* e *c2*. Para calcular *c1* utilizou-se a função `Compress` que irá comprimir o vetor $u = A^T * r + e1$ de forma a emilinar bits de *lower-order*. Para gerar a componente *c2* utilizou-se novamente a função *Compress* que irá comprimir o vetor $v = t^T * r + e2 + Decompress(m,1)$, sendo t = $pk[0]$ e m = mensagem a enviar. Desta forma, geramos o ciphertext $c = (Compress(u, du),Compress(v,dv))$

**Decifra:** A função `decrypt` foi implementada com recurso ao algoritmo 6 do paper e permite decifrar o *ciphertext* que recebe como pârametro utilizando a chave privada. Este algoritmo começa por recalcular os vetores *u* e *v* através da função `Decompress` que serão utilizadas para gerar a mensagem original calculada apartir da expressão $m = Compress(v − s^T * u, 1)$, com a s = chave privada.

In [178]:
#Classe que implementa o KYBER_PKE
class KYBER_PKE:
    
    #Função de inicialização das variaveis a usar nos métodos
    def __init__(self):
        self.n, self.q, self.T, self.k, self.n1, self.n2, self.du, self.dv, self.Rq = self.setup()
    
    #Parâmetros da técnica KYBER 512
    def setup(self):
        
        n = 256
        q = 7681
        k = 2
        n1 = 3
        n2 = 2
        du = 10
        dv = 4
        
        Zq.<w> = GF(q)[]
        fi = w^n + 1
        Rq.<w> = QuotientRing(Zq ,Zq.ideal(fi))
        
        T = NTT(n,q)
        
        return n,q,T,k,n1,n2,du,dv,Rq
    
    #Função auxiliar que transforma bytes em bits
    def bytesToBits(self, bytearr):
        bitarr = []
        
        for elem in bytearr:
            
            bitElemArr=[]

            for i in range(0,8):
                
                bitElemArr.append(mod(elem//2**(mod(i,8)),2))
                
                for i in range(0,len(bitElemArr)):
                    
                    bitarr.append(bitElemArr[i])
                    
        return bitarr
    
    #Função Hash que devolve um par
    def G(self, h):
        
        digest = hashes.Hash(hashes.SHA3_512())
        digest.update(bytes(h))
        g = digest.finalize()
        
        return g[:32],g[32:]
    
    #Função extendable output function (XOF)
    def XOF(self,b,b1,b2):
        
        digest = hashes.Hash(hashes.SHAKE128(int(self.q)))
        digest.update(b)
        digest.update(bytes(b1))
        digest.update(bytes(b2))
        m = digest.finalize()
        return m
    
    #Função pseudorandom function (PRF)
    def PRF(self,b,b1): 
        
        digest = hashes.Hash(hashes.SHAKE256(int(self.q)))
        digest.update(b)
        digest.update(bytes(b1))
        return digest.finalize()
    
    #Função que faz sampling dos elementos em Rq - Algoritmo 1
    def Parse(self,b):
        
        coefs=[]
        
        i = 0
        j = 0
        
        while j < self.n:
            d1 = b[i] + 256 * mod(b[i+1],16)
            d2 = b[i+1]//16 + 16 * b[i+2]
            
            if d1 < self.q :
                coefs.append(d1)
                j = j+1
            if d2 < self.q and j<self.n:
                coefs.append(d2)
                j = j+1
            i = i+3
            
        return self.Rq(coefs)
    
    
    #Função que faz sampling dos elementos de uma distribuição binomial - Algoritmo 2
    def CBD(self,arrayB,nn):
        
        f=[0]*self.n
        
        bitArray = self.bytesToBits(arrayB)
        
        for i in range(256):
            
            a = 0
            b = 0
            
            for j in range(nn):
                
                a += bitArray[2*i*nn + j]
                b += bitArray[2*i*nn + nn + j]

            f[i] = a-b
        
        return self.Rq(f)
    
    #Função que apartir de bytes gera um polinómio f pertencente a Rq
    def Decode(self,arrayB,l):
        
        f = []
        
        bitArray = self.bytesToBits(arrayB)
        
        for i in range(len(arrayB)):
            
            fi = 0
            
            for j in range(l):
            
                fi += int(bitArray[i*l+j]) * 2**j
            
            f.append(fi)
            
        return self.Rq(f)
    
    #Função elimina alguns low-order bits no x
    def Compress(self,x,d) :
    
        coefs = x.list()
    
        newCoefs = []
    
        for coef in coefs:
            new = mod(round( int(2 ** d) / self.q * int(coef)), int(2 ** d))
            newCoefs.append(new)
        
        return self.Rq(newCoefs)
    
    #Função repõem o x parcialmente
    def Decompress(self,x,d) :
        
        coefs = x.list()
    
        newCoefs = []
    
        for coef in coefs:
            new = round(self.q / (2 ** d) * int(coef))
            newCoefs.append(new)
        
        return self.Rq(newCoefs)
    
    #FUNÇÃO: Gera a chave pública e privada a ser utilizada no processo de cifra e decifra. 
    def keyGen(self):
        
        mtx = MyMatrix()
        
        # d ← B32
        d = bytearray(os.urandom(32))
        
        # (ρ, σ) := G(d)
        ro,sigma = self.G(d)
        
        # N := 0
        N = 0
        
        # Generate matrix Â ∈ Rq in NTT domain
        A = []
        
        for i in range(self.k):
            A.append([])
            for j in range(self.k):
                index = i*(self.k)+j
                A[i].append(self.T.ntt(self.Parse(self.XOF(ro,j,i))))
        
        
        # Sample s ∈ Rq from Bη1
        s = []
                
        for i in range(self.k):
            
            s.insert(i,self.CBD(self.PRF(sigma,N), self.n1))
            N = N+1
            
        #Sample e ∈ Rq from Bη1
        e = []
            
        for i in range(self.k):
            
            e.insert(i,self.CBD(self.PRF(sigma,N), self.n1))
            N = N+1

            
        # ŝ := NTT(s), ê := NTT(e)
        for i in range(self.k) :
            s[i] = self.T.ntt(s[i])
            e[i] = self.T.ntt(e[i])
            
        # t1 := Â ◦ ŝ
        tAux = mtx.multMatrixVector(A,s,self.k,self.n)
        # t := t1 + ê
        t = mtx.sumMatrix(tAux,e, self.n)
        
        # pk := As + e
        pk = t, ro
        # sk := s
        sk = s
        
        return pk,sk
    
    #FUNÇÃO: Cifra uma mensagem. 
    def encrypt(self,pk, m ,r):
        
        mtx = MyMatrix()
        
        # N := 0
        N = 0
        
        t, ro = pk
        
        # Generate matrix Â ∈ Rq in NTT domain
        transposeA = []
    
        for i in range(self.k):
            
            transposeA.append([])
            
            for j in range(self.k):
                
                transposeA[i].append(self.T.ntt(self.Parse(self.XOF(ro,i,j))))
        
        # Sample r ∈ Rq from Bη1, ^r:= NTT(r)
        rr = []
    
        for i in range(self.k):
            rr.insert(i,self.T.ntt(self.CBD(self.PRF(r, N), self.n1)))
            N += 1
        
        # Sample e1 ∈ Rq from Bη2
        e1 = []
    
        for i in range(self.k):
            e1.insert(i,self.CBD(self.PRF(r, N), self.n2))
            N += 1
        
        # Sample e2 ∈ Rq from Bη2
        e2 = self.CBD(self.PRF(r, N), self.n2)
        
        #Â ◦ r̂
        uAux = mtx.multMatrixVector(transposeA, rr, self.k, self.n)
        
        #NTT-1( Â ◦ r̂)
        uAux2 = []
        for i in range(len(uAux)) :
            uAux2.append(self.T.invNtt(uAux[i]))
        
        # u := NTT( Â ◦ r̂) + e1
        uAux3 = mtx.sumMatrix(uAux2,e1, self.n)
        
        u = []
        for i in range(len(uAux3)) :
            u.append(self.Rq(uAux3[i]))
            
        #t̂ ◦ r̂
        #t = [] + t
        vAux = mtx.multMatrix(t,rr,self.n)
        
        #NTT-1( t̂ ◦ r̂)
        vAux1 = self.T.invNtt(vAux)
        #NTT-1( t̂ ◦ r̂) + e2
        vAux2 = self.Rq(mtx.sumValue(vAux1,e2, self.n))
        
        #Decompress(m, 1)
        m1 = self.Decompress(m, 1)
        # v := NTT-1( t̂ ◦ r̂) + e2 + Decompress(m, 1)
        v = self.Rq(mtx.sumValue(vAux2,m1,self.n))
        
        # Compress(u, du)
        c1 = []
        
        for i in range(len(u)):
            c1.append(self.Compress(u[i],self.du))
        
        
        # Compress(v, dv)
        c2 = self.Compress(v,self.dv)
        
        return (c1,c2)
    
    
    def decrypt(self,sk, c):
        
        mtx = MyMatrix()
        
        c1, c2 = c
        
        #Decompress(c1,du)
        u = []
        
        for i in range(len(c1)):
            u.append(self.Decompress(c1[i],self.du))
        
        #Decompress(c2,dv)
        v = self.Decompress(c2,self.dv)

        s = sk
        
        #NTT(u)
        u1 = []
        for i in range(len(u)) :
            u1.append(self.T.ntt(u[i]))
        
        #ŝ ◦ NTT(u)
        mAux = mtx.multMatrix(s,u1,self.n)
        
        #v - NTT-1(ŝ ◦ NTT(u))
        mAux1 = mtx.subValue(v,self.T.invNtt(mAux), self.n)
        
        #Compress(v − NTT-1(ŝ ◦ NTT(u)), 1)
        m = self.Compress(self.Rq(mAux1), 1)
        
        return m

#### Cenário de teste
De seguida apresentamos o cenário de teste para o PKE-IND-CPA, sendo que a função `Decode` que é apresentada na especificação do KYBER e implementada na classe KYBER_PKE será utilizada para transformar um byte array que representa a mensagem a enviar num polinómio $f ∈ Rq$ para ser utilizado na função *Decompress* de forma a conseguir implementar o algoritmo *encrypt* que é apresentado na especificação.  

In [179]:
kyber = KYBER_PKE()

public, private = kyber.keyGen()

m = "Mensagem a enviar"
print("MENSAGEM ORIGINAL: ")
print(m)

ciphertext = kyber.encrypt(public, kyber.Decode(m.encode(),1), os.urandom(32))
print("CIPHERTEXT: ")
print(ciphertext)

plaintext = kyber.decrypt(private, ciphertext)
print("DECRYPT TEXT: ")
print(plaintext)

print("**** ENCRYPT e DECRYPT funcinou? ****")
print(plaintext == kyber.Decode(m.encode(),1))


MENSAGEM ORIGINAL: 
Mensagem a enviar
CIPHERTEXT: 
([420*w^255 + 708*w^254 + 595*w^253 + 999*w^252 + 856*w^251 + 151*w^250 + 97*w^249 + 387*w^248 + 305*w^247 + 619*w^246 + 133*w^245 + 34*w^244 + 286*w^243 + 330*w^242 + 604*w^241 + 818*w^240 + 522*w^239 + 244*w^238 + 755*w^237 + 196*w^236 + 44*w^235 + 295*w^234 + 833*w^233 + 570*w^232 + 541*w^231 + 843*w^230 + 464*w^229 + 595*w^228 + 172*w^227 + 327*w^226 + 340*w^225 + 761*w^224 + 342*w^223 + 865*w^222 + 899*w^221 + 140*w^220 + 308*w^219 + 990*w^218 + 564*w^217 + 449*w^216 + 984*w^215 + 405*w^214 + 798*w^213 + 492*w^212 + 8*w^211 + 901*w^210 + 29*w^209 + 217*w^208 + 487*w^207 + 281*w^206 + 260*w^205 + 674*w^204 + 469*w^203 + 913*w^202 + 1015*w^201 + 252*w^200 + 102*w^199 + 139*w^198 + 955*w^197 + 183*w^196 + 569*w^195 + 684*w^194 + 976*w^193 + 674*w^192 + 890*w^191 + 594*w^190 + 563*w^189 + 966*w^188 + 77*w^187 + 731*w^186 + 696*w^185 + 426*w^184 + 236*w^183 + 986*w^182 + 906*w^181 + 1000*w^180 + 401*w^179 + 454*w^178 + 958*w^177 + 645*

### Implementação KEM (IND-CPA)

Após a implementação do PKE-IND-CPA procedeu-se ao desenvolvimento do KEM que se encontra definido na classe KYBER_KEM. Tal como era pedido no enunciado, esta implementação do KEM consistiu no desenvolvimento de funções que permitissem uma segurança IND-CPA, ou seja, segurança contra ataques *Chosen Plaintext Attack*. Posto isto foram desenvolvidas três funções principais:

**Geração do par de chaves:** A função `keygen` tem como objetivo a criação do par de chaves pública e secreta que vão ser importantes no encapsulamento e desencapsulamento da chave. Simplesmente, foi utilizada a função *keygen* definida na classe KYBER_PKE.

**Encapsulamento da chave:** A função `encapsulate` recebe como *input* a chave pública e calcula a *hash* de um valor nonce aleatório *m ∈ Rq* utilizando a função `h` que irá representar a chave secreta *key*. De seguida fazemos a cifragem recorrendo à função *encrypt* da classe KYBER_PKE utilizando a variável *m* e a chave pública recebida como *input*. Desta função é retornado o criptograma *c* e a chave *key*.

**Desencapsulamento da chave:** A função `decapsulate` recebe como argumentos o criptograma e a chave secreta. De seguida, recorremos à função desenvolvida na classe KYBER_PKE para fazer a decifragem do criptograma. De seguida revela-se da chave secreta através da *hash* do resultado da decifragem. Caso a chave seja igual à gerada no processo de encapsulamento então a chave foi desencapsulada com sucesso. 

In [180]:
class KYBER_KEM:
    
    #Função de inicialização das variaveis a usar nos métodos
    def __init__(self):
        self.pke = self.setup()
        
    #Parâmetros da técnica KYBER 512
    def setup(self):
        
        pke = KYBER_PKE()
        
        return pke
    
    def H(self,b):
        
        r = hashes.Hash(hashes.SHA3_256())
        r.update(b)
        return r.finalize()
    
    def encapsulate(self,pk):
        
        #nonce aleatório
        m = self.pke.Rq([choice([0, 1]) for i in range(kyber.n)]) 
        key = self.H(dumps(m))
        
        r = os.urandom(256)
        c = self.pke.encrypt(pk,m,r)
        
        return c, key
    
    def decapsulate(self,c,sk):
        
        m = self.pke.decrypt(sk,c)
        
        key = self.H(dumps(m))
        
        return key
        
        
    def keygen(self):
        
        pk,sk = self.pke.keyGen()
        
        return pk,sk

#### **Cenário de Teste**

In [181]:
kem = KYBER_KEM()

pk, sk = kem.keygen()

c, k = kem.encapsulate(pk)
print("CIPHERTEXT: ")
print(c)
print()
print("ENCAP KEY: ")
print(k)

k2 = kem.decapsulate(c,sk)
print("DECAP KEY: ")
print(k2)

print("**** ENCAP e DECAP funcinou? ****")
print(k2 == k)


CIPHERTEXT: 
([315*w^255 + 591*w^254 + 50*w^253 + 481*w^252 + 87*w^251 + 523*w^250 + 971*w^249 + 994*w^248 + 793*w^247 + 167*w^246 + 648*w^245 + 836*w^244 + 950*w^243 + 293*w^242 + 604*w^241 + 186*w^240 + 896*w^239 + 187*w^238 + 213*w^237 + 859*w^236 + 207*w^235 + 509*w^234 + 276*w^233 + 959*w^232 + 448*w^231 + 862*w^230 + 912*w^229 + 347*w^228 + 877*w^227 + 506*w^226 + 335*w^225 + 403*w^224 + 672*w^223 + 42*w^222 + 349*w^221 + 362*w^220 + 560*w^219 + 983*w^218 + 826*w^217 + 120*w^216 + 734*w^215 + 370*w^214 + 540*w^213 + 304*w^212 + 1012*w^211 + 325*w^210 + 141*w^209 + 245*w^208 + 779*w^207 + 869*w^206 + 363*w^205 + 824*w^204 + 118*w^203 + 469*w^202 + 450*w^201 + 398*w^200 + 60*w^199 + 770*w^198 + 334*w^197 + 75*w^196 + 640*w^195 + 498*w^194 + 31*w^193 + 7*w^192 + 476*w^191 + 834*w^190 + 213*w^189 + 648*w^188 + 92*w^187 + 747*w^186 + 537*w^185 + 130*w^184 + 856*w^183 + 553*w^182 + 875*w^181 + 28*w^180 + 703*w^179 + 141*w^178 + 48*w^177 + 563*w^176 + 689*w^175 + 833*w^174 + 732*w^173 +

### Implementação PKE (IND-CCA)

Tal como é pedido no enunciado, na classe abaixo é implementado o algoritmo PKE-IND-CCA que irá cifrar uma mensagem utilizando para isso as transformação FOT (Fujisaki-Okamoto) para assim transformar um PKE-IND-CPA em PKE-IND-CCA. Assim sendo, será necessário implementar 3 funcionalidades principais:

**Geração do par de chaves:** A função de geração de chaves será dada pela função *keyGen* da classe KYBER_PKE

**Cifra:** Para implementação da função `encryptCCA` implementou-se as transformações FOT segunda a expressão:
  $$E(x)\;\equiv\;\vartheta\,r\gets h\,\centerdot\,\vartheta\,y \gets x\oplus g(r)\,\centerdot\,\vartheta\,c\gets f(r,h(r\,\|\,y))\,\centerdot\,(y\,,\,c)$$
Nesta função começamos por gerar um valor aleatório $r ∈ Rq$ e a sua hash apartir da função `g`. Este valor será que será utilizado juntamente, com a mensagem a cifrar $x$, na operação `xor` para formar a variariavel $y$. Esta variável será concatenada com o valor de $r$ e assim formar a *hash* $a = g(r\,\|\,y)$. É de notar que tanto a função *h* como a função *g* que aparecem na expressão acima são implementas utilizando a mesma função hash. Tanto a variavel $a$ com a $r$ serão utilizadas pela função $f$ que corresponde à função *encrypt* implementada na classe KYBER_PKE. No final obtemos os criptogramas parciais $c$ e $y$ com $c = f(r,a)\$. 

**Decifra:** Para implementação da função `decryptCCA` implementou-se as transformações FOT segunda a expressão:
$$D(y,c)\;\equiv\;\vartheta\,r \gets D(c)\,\centerdot\,\mathsf{if}\;\;c\neq f(r,h(r\|y))\;\;\mathsf{then}\;\;\bot\;\mathsf{else}\;\;y\oplus g(r)$$
Este algoritmo começa por decifrar o criptograma parcial $c$ utilizando para isso a função *decrypt* da classe KYBER_PKE, devolvendo como resultado o valor de $r$. Este valor vai ser novamente utilizado juntamente com a variável $y$ para calcular a variável $aux = f(r,h(r\|y))$, sendo esta função $f$ dada pela função *encrypt* da classe KYBER_PEK. De seguida, confirma-se se o processo de decifra correr como era esperado comparando o valor de $c$ e de $aux$. Caso seja igual, então significa que a operação correr bem e determinamos a mensagem que foi cifrada através da função *xor*. Caso não seja igual, então a operação não teve sucesso e por isso o processo de decifra falhou.

In [182]:
class KYBER_PKE_CCA:
    
    #Função de inicialização das variaveis a usar nos métodos
    def __init__(self):
        self.pke, self.pk, self.sk = self.setup()
        
    #Parâmetros da técnica KYBER 512
    def setup(self):
        
        pke = KYBER_PKE()
        
        pk, sk = pke.keyGen() 
        
        return pke,pk,sk
    
    #FUNÇÃO: Operação de XOR
    def xor(self,key, text):
    
        return bytes(a ^^ b for a, b in zip(key, text))
    
    #Função de hash h e g
    def g(self,b):
        
        r = hashes.Hash(hashes.SHA3_256())
        r.update(b)
        return r.finalize()
    
    #Função de encrypt para PKE-CCA
    def encryptCCA(self,x):
        
        # ϑr←h : valor aleatório a pretencer ao anel Rq
        r = self.pke.Rq([choice([0, 1]) for i in range(self.pke.n)])
        
        # ϑy← x ⊕ g(r)
        y = self.xor(x, self.g(bytes(r)))
        
        # c←f(r,h(r∥y)), f = encrypt do KYBER_PKE
        c = self.pke.encrypt(self.pk,r,self.g(bytes(r)+y))
        
        return (y, c)
    
    #Função de decrypt para PKE-CCA
    def decryptCCA(self,y,c):
        
        # ϑr←D(c), D = decrypt do KYBER_PKE
        r = self.pke.decrypt(self.sk, c)
        
        # f(r,h(r∥y))
        aux = self.pke.encrypt(self.pk,r,self.g(bytes(r)+y))
        
        # c ≠ f(r,h(r∥y))
        if c[0] != aux[0]:
            
            # ⊥
            return None
        else:
            
            # y ⊕ g(r)
            return self.xor(y, self.g(bytes(r)))

#### **Cenário de Teste**

In [183]:
pkeCCA = KYBER_PKE_CCA()

m = "Mensagem a enviar em modo CCA"
print("MENSAGEM ORIGINAL: ")
print(m)

parcialCipher = pkeCCA.encryptCCA(m.encode())
print("CIPHERTEXT: ")
print(parcialCipher)

(y,c) = parcialCipher

result = pkeCCA.decryptCCA(y,c)

if result == None:
    print("**** ENCRYPT-CCA e DECRYPT-CCA não funcinou ****")
else:
    print("**** ENCRYPT-CCA e DECRYPT-CCA funcinou ****")
    print(result.decode())

MENSAGEM ORIGINAL: 
Mensagem a enviar em modo CCA
CIPHERTEXT: 
(b'\x8c\xccK\n\x90\xf6\xdd\x83a\xc0-\xd0uR\xe7;\xe9\xbf\x9fH\xf7\x94\xda\x80~\x9ei^"', ([383*w^255 + 388*w^254 + 697*w^253 + 153*w^252 + 635*w^251 + 549*w^250 + 778*w^249 + 255*w^248 + 851*w^247 + 77*w^246 + 949*w^245 + 1010*w^244 + 733*w^243 + 632*w^242 + 369*w^241 + 19*w^240 + 931*w^239 + 218*w^238 + 251*w^237 + 400*w^236 + 902*w^235 + 432*w^234 + 66*w^233 + 243*w^232 + 825*w^231 + 617*w^230 + 999*w^229 + 892*w^228 + 668*w^227 + 536*w^226 + 697*w^225 + 103*w^224 + 190*w^223 + 953*w^222 + 149*w^221 + 424*w^220 + 510*w^219 + 212*w^218 + 469*w^217 + 836*w^216 + 132*w^215 + 717*w^214 + 664*w^213 + 273*w^212 + 991*w^211 + 408*w^210 + 556*w^209 + 742*w^208 + 976*w^207 + 679*w^206 + 396*w^205 + 464*w^204 + 823*w^203 + 266*w^202 + 45*w^201 + 245*w^200 + 852*w^199 + 30*w^198 + 451*w^197 + 839*w^196 + 824*w^195 + 385*w^194 + 256*w^193 + 433*w^192 + 442*w^191 + 169*w^190 + 534*w^189 + 751*w^188 + 266*w^187 + 787*w^186 + 957*w^185 + 

**** ENCRYPT-CCA e DECRYPT-CCA funcinou ****
Mensagem a enviar em modo CCA
